# Modélisation des forces perturbatrices en orbite terrestre basse
***Application à la Station Spatiale Internationale***

Lancée en 1998, la Station spatiale internationale (ISS) est le plus grand des objets artificiels placés en orbite terrestre, pour une longueur de $110m$, une largeur de $74m$ et une hauteur de $30m$. Elle est avant tout un laboratoire de recherche qui permet à des astronautes et des scientifiques du monde entier de mener des expériences dans l'espace dans des conditions de microgravité, et doit rester en service pour une durée de 30 ans, c'est à dire jusqu'à 2028.

Au-delà des considérations géostratégiques, diplomatiques, et des objectifs scientifiques qui lui sont attachés, l'ISS est un objet céleste en orbite basse (environ $450km$ au-dessus de la Terre), et qui possède donc une trajectoire, une vitesse, et de nombreuses caractéristiques physiques particulièrement intéressantes à étudier. Sa trajectoire est en effet calculée minutieusement, dans le but de la garder en orbite, et de multiples variables viennent la perturber, comme la gravitation terrestre, la force de traînée atmosphérique, ou encore les forces de gravitation exercées par le Soleil et la Lune. 

Dès lors, le but de ce projet est la modélisation des forces perturbatrices qui viennent s'excercer sur un satellite en orbite basse, et plus particulièrement sur l'ISS. Nous allons donc chercher à modéliser les forces prédominantes agissant sur les objets placés en orbite basse, en sélectionnant celles qui nous paraissent pertinentes. Au travers de nos recherches, nous tenterons d'estimer l'impact de ces forces sur la trajectoire de l'ISS. En effet, le principe fondamental de la dynamique (PFD) nous indique que la somme des forces agissant sur un corps est égale à la masse du corps multipliée par son accélération. En intégrant, nous obtiendrons alors un modèle de la vitesse, puis de la trajectoire de l'ISS, en tenant compte des paramètres initiaux. Exprimée en coordonnées cylindriques, basées sur la longitude et la latitude, nous comparerons alors notre modèle aux données collectées par la NASA, afin de conclure sur la pertinence de notre modèle, et sur la pertinence de modéliser uniquement forces que nous avons sélectionnées. Nous serons alors en mesure de présenter sur les difficultés que nous avons rencontrées, les limites de nos travaux, ainsi que l'intérêt de ce projet en termes d'apprentissage.


## Modélisation

En tenant compte de l'ordre de grandeur de chaque force excercée sur un satellite en orbite basse, nous en avons sélectionnées les suivantes dans un objectif de simplification : 

- Force géopotentielle
- Force de traînée atmosphérique
- Force gravitationnelle de la Lune
- Force gravitationnelle du Soleil

### Force géopotentielle

Si l'on considère que la masse de la Terre n'est pas uniformément répartie dans une sphère parfaite, on peut choisir d'intégrer le potentiel $U$ de la gravité terrestre pour modéliser sa force d'attraction sur l'ISS.  
On a 
$\ddot{r}$ = $\nabla U$, avec $U = GM\frac{1}{r}$

Si on généralise l'expression à une masse arbitraire, 
$ U = $$\int \frac{\rho(s)d^3s}{|r - s|}$$

avec ${\rho(s)}$ la densité à un point $s$ au sein de la Terre, et $|r - s|$ la distance du satellite au point $s$.

En développant avec la méthode des polynômes de Legendre, $$
\ddot{r} = \nabla\frac{GM}{r} * \sum_{n=0}^{∞} \sum_{m=0}^{n} \frac{R^n}{r^n} \bar{P}_{nm}(sin\phi)(\bar{C}_{nm} cos(m\lambda) + \bar{S}_{nm} sin(m\lambda)) 
$$

Ce qui nous donne, si développé aux coefficients d'ordre 2,

$$
\boxed{
\ddot{r} = \nabla\frac{GM}{r} + \nabla\frac{GM}{r} \frac{R^2}{r^2} \sqrt{5} \frac{1}{2}(3 sin^2\phi - 1) \bar{C}_{2 ,0} + \nabla\frac{GM}{r}\frac{R^2}{r^2} 3cos^2\phi \sqrt \frac{5}{12} (\bar{C}_{2,2} cos(2\lambda) + \bar{S}_{2,2} sin(2\lambda)) 
}
$$

C'est l'expression de la force géopotentielle qui s'exerce sur le satellite, avec $r$ la distance du satellite au centre de la Terre, $R$ le rayon ce cette dernière.

La méthode générale est expliquée plus en détail dans le **Notebook *Force de gravitation terrestre***.


## Données sources
[TODO] Explication des données NASA

## Intégration numérique
[TODO] Explication de l'intégration des EDO

## Visualisation des résultats
[TODO] Rédaction

## Discussion
[TODO] Observations et limites du modèle

## Conclusion
Apprentissage et ouvertures de cette modélisation